# Prophet-EMD Hybrid Time Series Forecasting

In this notebook, we build a **hybrid forecasting model** by combining:

- **Empirical Mode Decomposition (EMD)** to break down the series into simpler components (IMFs),
- **Prophet** forecasting models for each component,
- and **aggregating** the results to make a final prediction.

We also evaluate the model performance using:

- Mean Absolute Percentage Error (MAPE),
- Root Mean Squared Error (RMSE)

---

> **Goal**: Enhance forecasting accuracy by modeling each intrinsic pattern separately.


## Import libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_percentage_error , mean_squared_error
from PyEMD import EMD
from PyEMD import CEEMDAN
from prophet import Prophet

import nolds
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.seasonal import DecomposeResult

import pandas as pd
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro

## Read Data

In [ ]:
path = #path/to/dataset
df = pd.read_csv(path)
df.head()

test_size = 12
train_set = df[:len(df)-test_size]
test_set = df[len(df)-test_size:]

ds = df.ds

## Feature Extraction

We extract key time series characteristics using Python and R, including:
- Chaos level (Lyapunov exponent)
- Seasonal and trend strength (via STL)
- Statistical features (via R’s `tsfeatures`)

In [ ]:
# Chaos Analysis: Lyapunov Exponent and STL Decomposition ---
# Calculate Lyapunov exponent to detect chaos (positive value → chaotic system)
h = nolds.lyap_r(df.y.values[:-test_size])
print("lyap", h)

# Perform STL decomposition (monthly frequency assumed)
res = STL(df.y, period=12).fit()
print(res.seasonal)
res.plot()
plt.show()

In [ ]:
# Setup for Seasonal Strength Feature Extraction using R
# Install required R packages via R magic in Jupyter to enable time series feature extraction.
# Packages include:
# - tsfeatures: for computing time series features such as seasonal strength.
# - forecast: supports time series decomposition and forecasting.
# - dplyr and tidyr: for data manipulation within the R environment.
%load_ext rpy2.ipython

%%R
install.packages("tsfeatures", repos='http://cran.r-project.org')
install.packages("forecast", repos='http://cran.r-project.org')
install.packages("dplyr", repos='http://cran.r-project.org')
install.packages("tidyr", repos='http://cran.r-project.org')


In [ ]:
# Data Preparation for R-based Feature Extraction ---
# Extract the 'y' column, and convert it to an R time series object with monthly frequency.

# Enable automatic conversion between Pandas and R data frames
pandas2ri.activate()

# Extract the target time series column ('y') from training data
x = df['y'].iloc[:-test_size].values

# Transfer the time series to R environment
ro.globalenv['x'] = ro.FloatVector(x)

# Convert to R time series object with monthly frequency
ro.r('x <- ts(x, start = c(1998, 1), frequency = 12)')


## prophet-EMD

In [ ]:
## decomposition 
emd = EMD()
imfs = emd.emd(np.array(list(train_set.y.values)))
print(imfs.shape)

forecast_df = pd.DataFrame()
forecast_df['ds'] = ds
forecast_df['yhat'] = 0

## training
for imf in imfs:
  temp = pd.DataFrame(imf)
  temp['ds'] = ds
  temp.columns = ['y', 'ds']
  model = Prophet()
  model.fit(temp)
  future = model.make_future_dataframe(test_size, freq="MS")
  forecast = model.predict(future)
  forecast_df["yhat"] += forecast['yhat']

## evaluation 
mape = mean_absolute_percentage_error(test_set, forecast_df.yhat[-test_size:])
rmse = np.sqrt(mean_squared_error(test_set, forecast_df.yhat[-test_size:]))

print(f" EMD-Prophet mape : {mape}")
print(f" EMD-Prophet rmse : {rmse}")

## Plotting




## Prophet-CEEMDAN

In [ ]:
## decomposition
ceemd = CEEMDAN()
ceemd_imfs = ceemd.ceemdan(np.array(list(train_set.y.values)))
print(f"ceemdan imfs shape : {ceemd_imfs.shape}")

## training
forecast_ceemdan_df = pd.DataFrame()
forecast_ceemdan_df['ds'] = ds
forecast_ceemdan_df['yhat'] = 0

for imf in ceemd_imfs:
  temp = pd.DataFrame(imf)
  temp['ds'] = ds
  temp.columns = [ 'y', 'ds']
  model = Prophet(seasonality_mode="multiplicative")
  model.fit(temp)
  future = model.make_future_dataframe(test_size, freq="MS")
  forecast = model.predict(future)
  forecast_ceemdan_df["yhat"] += forecast['yhat']
    
## evaluation
ceemdan_mape = mean_absolute_percentage_error(test_set, forecast_ceemdan_df.yhat[-test_size:])
ceemdan_rmse = np.sqrt(mean_squared_error(test_set, forecast_ceemdan_df.yhat[-test_size:]))

print(f" CEEMDAN-Prophet mape : {ceemdan_mape}")
print(f" CEEMDAN-Prophet rmse : {ceemdan_rmse}")

## Plotting